Sequential Chats and Customer Onboarding

## Setup

In [1]:
llm_config={"model": "gpt-3.5-turbo"}

In [2]:
from autogen import ConversableAgent

## Creating the needed agents

In [3]:
onboarding_personal_information_agent = ConversableAgent(
    name="Onboarding Personal Information Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [4]:
onboarding_topic_preference_agent = ConversableAgent(
    name="Onboarding Topic preference Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [5]:
customer_engagement_agent = ConversableAgent(
    name="Customer Engagement Agent",
    system_message='''You are a helpful customer service agent
    here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

In [6]:
customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

## Creating tasks

In [7]:
chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": 
            "Hello, I'm here to help you get started with our product."
            "Could you tell me your name and location?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'name': '', 'location': ''}",
        },
        "max_turns": 2,
        "clear_history" : True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message": 
                "Great! Could you tell me what topics you are "
                "interested in reading about?",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history" : False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

## Start the onboarding process

In [8]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Personal Information Agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product.Could you tell me your name and location?

--------------------------------------------------------------------------------


/usr/local/lib/python3.11/site-packages/autogen/agentchat/chat.py:47: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


Provide feedback to Onboarding Personal Information Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: Aice
customer_proxy_agent (to Onboarding Personal Information Agent):

Aice

--------------------------------------------------------------------------------
Onboarding Personal Information Agent (to customer_proxy_agent):

Thank you for providing your name. Could you also share your location with me?

--------------------------------------------------------------------------------
Provide feedback to Onboarding Personal Information Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: San Fransisco
customer_proxy_agent (to Onboarding Personal Information Agent):

San Fransisco

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

******************************************************

## Print out the summary

In [9]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

{'name': 'Aice', 'location': 'San Francisco'}


Aice from San Francisco is interested in reading about AI jobs in California.


Aice from San Francisco is interested in reading about AI jobs in California, particularly in the Silicon Valley area. The tech hub offers various opportunities in the field of AI for talented individuals like Aice. Additionally, AI systems have been trained to write jokes, such as the one shared about a robot going on a diet because it had too many bytes.




## Print out the cost

In [10]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 0.000138, 'gpt-3.5-turbo-0125': {'cost': 0.000138, 'prompt_tokens': 186, 'completion_tokens': 30, 'total_tokens': 216}}, 'usage_excluding_cached_inference': {'total_cost': 0.000138, 'gpt-3.5-turbo-0125': {'cost': 0.000138, 'prompt_tokens': 186, 'completion_tokens': 30, 'total_tokens': 216}}}


{'usage_including_cached_inference': {'total_cost': 5.6e-05, 'gpt-3.5-turbo-0125': {'cost': 5.6e-05, 'prompt_tokens': 67, 'completion_tokens': 15, 'total_tokens': 82}}, 'usage_excluding_cached_inference': {'total_cost': 5.6e-05, 'gpt-3.5-turbo-0125': {'cost': 5.6e-05, 'prompt_tokens': 67, 'completion_tokens': 15, 'total_tokens': 82}}}


{'usage_including_cached_inference': {'total_cost': 0.000614, 'gpt-3.5-turbo-0125': {'cost': 0.000614, 'prompt_tokens': 394, 'completion_tokens': 278, 'total_tokens': 672}}, 'usage_excluding_cached_inference': {'total_cost': 0.000614, 'gpt-3.5-turbo-0125': {'cost': 0.000614, 'prompt_tokens': 394, 'completion_toke